In [5]:

# coding: utf-8

# ## Imports

# In[8]:

import numpy as np
import pandas as pd
from collections import Counter
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from xgboost.sklearn import XGBClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Imputer
from sklearn.metrics import log_loss, accuracy_score
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score
from IPython.display import display, HTML
from sklearn.externals import joblib
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import f1_score, make_scorer
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', 500)
seed = 1


# ## Load dataset / Preprocessing

# In[9]:




input_file = "../data/ait_result_dataset.V38.csv"

df = pd.read_csv(input_file, header = 0)
print("DF original: "+str(df.shape))
#df=df.replace('NaN',np.nan)
before=df.shape[1]
df=df.dropna(axis=1, how='all')
after=df.shape[1]
#print(str(before-after)+ " NaN columns dropped")


#df=df.dropna(axis=1, thresh=len(df) - 10)
#df=df.dropna(axis=0, how='all')
print("DF after dropna: "+str(df.shape))
df=df.replace(np.nan,0)
#display(df)
numpy_array = df.as_matrix()

n_features=numpy_array.shape[1]
X=numpy_array[:,0:n_features-1]
y=numpy_array[:,n_features-1]

#display(pd.DataFrame(X))
#display(pd.DataFrame(y))

print("X:"+str(X.shape))
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(X)
X = imp.transform(X)
print("X after imputer:"+str(X.shape))

#display(pd.DataFrame(X))


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

print("Train label distribution:")
print(Counter(y_train))

print("\nTest label distribution:")
print(Counter(y_test))

print("\nTrain dataset contains {0} rows and {1} columns".format(X_train.shape[0], X_train.shape[1]))
print("Test dataset contains {0} rows and {1} columns".format(X_test.shape[0], X_test.shape[1]))

def decision_tree():
    # ## Decision tree

    # In[148]:

    decision_tree = DecisionTreeClassifier(random_state=seed) #,class_weight="balanced"

    # train classifier
    decision_tree.fit(X_train, y_train)

    # predict output
    decision_tree_y_pred  = decision_tree.predict(X_test)
    decision_tree_y_pred_prob  = decision_tree.predict_proba(X_test)

    # evaluation
    decision_tree_accuracy = accuracy_score(y_test, decision_tree_y_pred)
    decision_tree_logloss = log_loss(y_test, decision_tree_y_pred_prob)

    print("== Decision Tree ==")
    print("Accuracy: {0:.2f}".format(decision_tree_accuracy))
    print("Log loss: {0:.2f}".format(decision_tree_logloss))
    print("Number of nodes created: {}".format(decision_tree.tree_.node_count))
    print("F1: {:1.4f}".format(f1_score(y_test, adaboost_y_pred,labels=['A','O','N'],average='macro')))  


def adaboost():
    # ## Adaboost

    # In[91]:

    decision_tree = DecisionTreeClassifier(random_state=seed)
    adaboost = AdaBoostClassifier(
        base_estimator=decision_tree,
        algorithm='SAMME.R',
        n_estimators=100,
        random_state=seed,
        learning_rate=0.1)

    # train classifier
    adaboost.fit(X_train, y_train)

    # calculate predictions
    adaboost_y_pred = adaboost.predict(X_test)
    adaboost_y_pred_prob = adaboost.predict_proba(X_test)

    # evaluate
    adaboost_accuracy = accuracy_score(y_test, adaboost_y_pred)
    adaboost_logloss = log_loss(y_test, adaboost_y_pred_prob)

    print("== AdaBoost ==")
    print("Accuracy: {0:.2f}".format(adaboost_accuracy))
    print("Log loss: {0:.2f}".format(adaboost_logloss))
    print("F1: {:1.4f}".format(f1_score(y_test, adaboost_y_pred,labels=['A','O','N'],average='macro')))  


def rf():
    # ## Random Forest

    # In[17]:

    # Create a random forest Classifier. By convention, clf means 'Classifier'
    rf = RandomForestClassifier(n_jobs=-1, random_state=seed,n_estimators=100,class_weight="balanced")

    # Train the Classifier to take the training features and learn how they relate
    # to the training y (the species)
    rf.fit(X_train, y_train)

    # calculate predictions
    rf_y_pred = rf.predict(X_test)
    rf_y_pred_prob = rf.predict_proba(X_test)

    # evaluate
    rf_accuracy = accuracy_score(y_test, rf_y_pred)
    rf_logloss = log_loss(y_test, rf_y_pred_prob)

    print("== RandomForest ==")
    print("Accuracy: {0:.2f}".format(rf_accuracy))
    print("Log loss: {0:.2f}".format(rf_logloss))
    print("F1: {:1.4f}".format(f1_score(y_test, rf_y_pred,labels=['A','O','N'],average='macro')))  



def gbc():
    # ## Gradient Boosting Classifier

    # In[147]:

    gbc = GradientBoostingClassifier(
        max_depth=1,
        n_estimators=100,
        random_state=seed)
    gbc.fit(X_train, y_train)

    # make predictions
    gbc_y_pred = gbc.predict(X_test)
    gbc_y_pred_prob = gbc.predict_proba(X_test)

    # calculate log loss
    gbc_accuracy = accuracy_score(y_test, gbc_y_pred)
    gbc_logloss = log_loss(y_test, gbc_y_pred_prob)

    print("== Gradient Boosting ==")
    print("Accuracy: {0:.2f}".format(gbc_accuracy))
    print("Log loss: {0:.2f}".format(gbc_logloss))
    print("F1: {:1.4f}".format(f1_score(y_test, gbc_y_pred,labels=['A','O','N'],average='macro')))  



def xgboost():
    # ## XGBoost

    # In[183]:

    #params = {
    #    'objective': 'multi:softmax',
    #    'max_depth': 7,
    #    'learning_rate': 0.1,
    #    'silent': 1,
    #    'n_estimators': 200,
    #    'nthread': 4,
    #    'colsample_bytree': 0.8,
    #    'gamma':0,
    #       'subsample':1,
    #       'min_child_weight':1
    #}
    #84.53,83.85 (cv)
    
    params = {
        'objective': 'multi:softmax',
        'max_depth': 9,
        'learning_rate': 0.10836619673151071,
        'silent': 1,
        'n_estimators': 200,
        'nthread': 8,
        'colsample_bytree': 0.27958200096140395,
        'gamma':0.25,
        'subsample':0.93166895175080411,
        'min_child_weight':1,
        'colsample_bylevel':1,
        'max_delta_step':0,
        'missing':None,
        'reg_alpha':0
    }
    #84.33,
    bst = XGBClassifier(**params)
    #xgb_y_pred = bst.predict(X_test)

    #print("F1: {:1.4f}".format(f1_score(y_test, xgb_y_pred,labels=['A','O','N'],average='macro')))  

    #display(pd.DataFrame(xgb_y_pred))
    #correct = 0

    #for i in range(len(xgb_y_pred)):
    #    if (y_test[i] == xgb_y_pred[i]):
    #        correct += 1
    #acc = accuracy_score(y_test, xgb_y_pred)
    #print('Predicted correctly: {0}/{1}'.format(correct, len(xgb_y_pred)))
    #print('Error: {0:.4f}'.format(1-acc))
    #joblib.dump(bst, 'xgboost.pkl', compress = 1)
    scorer = make_scorer(f1_score, labels=['A','O','N'], average='macro')
    scores = cross_val_score(bst, X, y, cv=5,scoring=scorer)
    print("F1 scores:",scores)
    print("F1 mean:",np.mean(scores))
    joblib.dump(bst, 'xgboost.pkl', compress = 1)
    joblib.dump(scores, 'scores.pkl', compress = 1)

# In[4]:
def gridSearch():
    from sklearn.grid_search import GridSearchCV
    from sklearn.cross_validation import StratifiedKFold

    
    

    cv = StratifiedKFold(y, n_folds=5, shuffle=True, random_state=seed)


    # In[ ]:
 
    params_grid = {
       'gamma':[0,1,2],
        'min_child_weight': [0.1,1,20],
        'subsample':[0.8,0.9,1], 
        'colsample_bytree': [0.8,0.9,1]

    }


    params_fixed = {
        'objective': 'multi:softmax',
        'silent': 1,
        'max_depth': 7,
        'n_estimators': 200,
         'nthread': 4
    }


    scorer = make_scorer(f1_score, labels=['A','O','N'], average='macro')

    bst_grid = GridSearchCV(
        estimator=XGBClassifier(**params_fixed, seed=seed),
        param_grid=params_grid,
        cv=cv,
        scoring=scorer
    )



    bst_grid.fit(X, y)
    print(bst_grid.best_estimator_)
    joblib.dump(bst_grid, 'grid.pkl', compress = 1)

#gridSearch()
xgboost()

DF original: (8528, 397)
DF after dropna: (8528, 391)
X:(8528, 390)
X after imputer:(8528, 386)
Train label distribution:
Counter({'N': 4038, 'O': 1959, 'A': 592, '~': 233})

Test label distribution:
Counter({'N': 1012, 'O': 496, 'A': 147, '~': 51})

Train dataset contains 6822 rows and 386 columns
Test dataset contains 1706 rows and 386 columns
F1 scores: [ 0.84750944  0.86793837  0.84849727  0.79283666  0.85404859]
F1 mean: 0.842166064022


In [6]:
scores=joblib.load('scores.pkl')
print("F1:",np.mean(scores))

F1: 0.842166064022
